## Task: Part-of-speech tagging of Treaty Headnote
This script PoS-tags the treaty headnotes for the entire WTI master data. It uses Stanford's CoreNLP package and the Stanford POS tagger to compute the PoS-tags, and the resulting data is stored in a tab seperated file "./data/tagged_headnotes.csv".

Note that it will take some time, perhaps an hour, to execute the script. The script needs only to be run once, though, unless changes are made to any headnote or errors are found in the data.


In [ ]:
# CoreNLP Dependency Parsing. Start CoreNLP server (~roger/applications/stanford.../start-server.bash)
from nltk.tag.stanford import CoreNLPPOSTagger
if 'pos_tagger' not in globals():
    pos_tagger = nltk.tag.stanford.CoreNLPPOSTagger(url='http://127.0.0.1:9000', encoding='utf8')

def pos_tag_texts(pos_tagger, source_texts, done_texts):
    pending_keys = set(source_texts.index) - set(done_texts.keys())
    counter = 0
    for key in pending_keys:
        counter += 1
        text = source_texts.loc[key] or ''
        try:
            done_text = pos_tagger.tag(text.lower().split())
        except:
            print('Failed: {}[{}]'.format(key, text))
            done_text = []
        done_texts[key] = done_text
        if counter % 1000 == 0:
            print('Done: {} out of {}'.format(counter, len(pending_keys)))
    '''
    Add key and sequence number to each key's tuples
    '''  
    keyed_texts = {
        key: [ (key, i, w, p) for i, (w, p) in enumerate(done_texts[key]) ] for key in done_texts.keys()
    }
    '''
    Flatten list to list of tuples
    '''  
    keyed_texts_list = [ item for sublist in list(keyed_texts.values()) for item in sublist ]

    return keyed_texts_list
    
tagged_treaties = {}
headnotes = state.get_headnotes()
tagged_headnotes = pos_tag_texts(pos_tagger, headnotes, tagged_treaties)

df_tagged_headnotes = pd.DataFrame(tagged_headnotes, columns=['treaty_id', 'sequence_id', 'word', 'pos'])
df_tagged_headnotes.to_csv('./data/tagged_headnotes.csv', sep='\t')

# Failed: 134271[] Failed: 124126[] Failed: 133760[]

# pos_tagger.session.close()